목표

Aruomarker 생성 (기존 코드 활용)

Arucomarker 검출 (기존 코드 활용)

Arucomarker 카메라에서 인식

Arucomarker 상대 위치 인식

Arucomarker 정밀도 수정 연습

In [1]:
import numpy as np
import imutils
import cv2
import sys

In [2]:
# OpenCV에서 지원하는 ARUCO MARKER의 TYPE에 대한 딕셔너리
ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

In [3]:
def pose_esitmation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    '''
    frame - 카메라 또는 비디오에서 읽어온 이미지
    aruco_dict_type - 사용하는 MARKER의 TYPE (예. 앞서 설명한 코드의 ARUCO_DICT[aruco_type])
    matrix_coefficients - 카메라 Calibration 과정 후에 획득한 Intrinsic matrix (내부 행렬)
    distortion_coefficients - 카메라 Calibration 과정 후에 획득한 Distortion coefficients (왜곡 계수)
    return:
    frame - 읽어온 이미지 + MARKER의 축
    '''

    # 카메라 또는 비디오에서 읽어온 이미지를 흑백으로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 정의한 ARUCO MARKER의 TYPE을
    # 컴퓨터가 이해할 수 있는 형태의 언어로 변환하는 부분임
    cv2.aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    # 검출을 위한 파라미터 설정하는 부분임
    # 일반적으로 default 값을 그대로 사용함
    parameters = cv2.aruco.DetectorParameters_create()
    # MARKER 검출
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters)
        # cameraMatrix=matrix_coefficients,
        # distCoeff=distortion_coefficients)

    # 하나 이상의 MARKER가 검출됐을 때만 실행되는 부분
    if len(corners) > 0:
        for i in range(0, len(ids)):
            # 각 MARKER의 Pose를 측정 하고 rvec 과 tvec 으로 반환
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                       distortion_coefficients)
            # MARKER 테두리 그려주기 (삭제해도 됨)
            cv2.aruco.drawDetectedMarkers(frame, corners) 

            # MARKER 축 그려주기 (삭제해도 됨)
            cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)  

    return frame

In [4]:
# 인식할 MARKER의 TYPE을 정의해줌
aruco_type = "DICT_4X4_50"
print("[INFO] detecting '{}' tags...".format(aruco_type))
arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

[INFO] detecting 'DICT_4X4_50' tags...


In [5]:
import cv2

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = capture.read()
    # cv2.imshow("VideoFrame", frame)
    
    det_frame = pose_esitmation(frame, ARUCO_DICT[aruco_type], (0,0,0,0), (0,0,0,0))
    cv2.imshow("VideoFrame", det_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
capture.release()

NameError: name 'matrix_coefficients' is not defined